# Cyft Data Investigation

Goal:


#### Description:

_30-Day All-Cause Hospital Readmissions_ is a quality measure that many healthcare organizations use to track their performance. Lower readmission rates indicate better patient outcomes, while higher ones tend to indicate system problems that are negatively impacting patients. The goal of this exercise is to analyze a dataset that simulates hospitalizations for a geriatric patient population in 2015 and 2016 to predict __if a patient is likely to have a readmission based on the information available at the time of their initial admission.__

You have 3 hours to complete the exercise. If you don't get through all the objectives, that's OK. After 3 hours, please finish what you're working on and send in whatever code, analyses, and visualizations (such as images) you have available. Include comments documenting any assumptions you've made as well as other ideas you would have tried if you had more time.

Feel free to use the language and statistical/machine learning libraries that you are most comfortable with, and ask questions along the way if any clarifications are necessary.

In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, accuracy_score
from sklearn.cross_validation import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.grid_search import GridSearchCV
import seaborn as sns

In [3]:
Dxs_summarized = {'A41': 'Sepsis',
 'E11': 'diabetes mellitus',
 'E56': 'Vitamin deficiency',
 'E86': 'Dehydration',
 'F03': 'dementia',
 'F05': 'Delirium',
 'F19': 'Drug abuse',
 'G31': 'Degeneration of nervous system',
 'G89': 'Chronic Pain',
 'H53': 'Visual discomfort',
 'H91': 'Hearing loss',
 'I10': 'hypertension',
 'I50': 'Heart Failure',
 'I51': 'heart disease',
 'J18': 'Pneumonia',
 'J44': 'COPD',
 'J45': 'asthma',
 'M54': 'Radiculopathy/Panniculitis/Sciatica',
 'N18': 'Chronic kidney disease',
 'N39': 'Urinary incontinence',
 'R05': 'cough',
 'R26': 'abnormalities of gait and mobility',
 'R39': 'Urgency of urination',
 'R41': 'Cognitive functions and awareness symptoms',
 'R51': 'headache',
 'T88': 'Anesthesia Complication',
 'W19': 'Fall'}

In [4]:
def add_target(full):
    full['Readmitted'] = full.groupby(['Patient'])['Patient'].transform('count')
    full['Readmitted'] = full['Readmitted'].map({2:1,1:0})
    full['DaysSinceAdmission'] = full[full['Readmitted'] == 1].groupby(['Patient']).diff()['AdmitDate']
    full['<=30Days'] = (full['DaysSinceAdmission'] <= pd.Timedelta('30 days')).astype(np.int)
    full['WillBe<=30Days'] = full[full['Readmitted'] == 1].groupby('Patient').shift(-1)[['<=30Days']].fillna(0).astype(np.int)
    full['WillBe<=30Days'] = full['WillBe<=30Days'].fillna(0)
    full['<=30Days'] = full['WillBe<=30Days']
    full.drop(['Readmitted','DaysSinceAdmission','WillBe<=30Days'],axis=1,inplace=True)
    return full

In [5]:
def add_combined_dx_feats(full_orig):
    full_orig['PrimaryDx_Dx2'] = full_orig['PrimaryDx']+full_orig['Dx2']
    full_orig['PrimaryDx_Dx3'] = full_orig['PrimaryDx']+full_orig['Dx3']
    full_orig['Dx2_Dx3'] = full_orig['Dx2']+full_orig['Dx3']
    return full_orig

In [6]:
def add_month_feat(df):
    df['MonthAdmit'] = df['AdmitDate'].apply(lambda x: x.month)
    df['MonthAdmit'] = df['MonthAdmit'].map({1:'Jan',
                                            2:'Feb',
                                            3:'Mar',
                                            4:'Apr',
                                            5:'May',
                                            6:'Jun',
                                            7:'Jul',
                                            8:'Aug',
                                            9:'Sep',
                                            10:'Oct',
                                            11:'Nov',
                                            12:'Dec'})
    return df

In [7]:
def replace_nulls(full):
    full = full.replace('@NA',np.NaN) # replace @NA
    full = full.replace('',np.NaN) # didn't find any empty strings
    full = full.replace(np.NaN,'')
    return full

#### Load and transform data

Functions created in other eda notebook and copied over

In [95]:
admit = pd.read_csv('../Cyft/readmissions/admissions.csv')
claims = pd.read_csv('../Cyft/readmissions/claims.csv')

full = pd.merge(admit,claims,on=['Patient','AdmitDate'])
full['AdmitDate'] = pd.to_datetime(full['AdmitDate'])

full = replace_nulls(full)
full = add_target(full)
#full = add_combined_dx_feats(full)
full = add_month_feat(full)

In [96]:
full_orig = full.copy()

full = full.set_index('AdmitDate')

for col in [col for col in full.columns if full[col].dtype == 'object' and 'Patient' not in col]:
    dummies = pd.get_dummies(full[col],prefix=col)
    full.drop(col,axis=1,inplace=True)
    full = pd.concat([full,dummies],axis=1)    

In [97]:
train = full['2015']
test = full['2016']

train = train.drop(['Patient'],axis=1)
test = test.drop(['Patient'],axis=1)

print('train: {} rows and {} columns'.format(*train.shape))
print('test: {} rows and {} columns'.format(*test.shape))

train: 2856 rows and 49 columns
test: 2938 rows and 49 columns


In [98]:
print('Accuracy baseline is: {:.2f}%'.format(100*(1-full['<=30Days'].mean())))


Accuracy baseline is: 76.44%


In [99]:
# X_train = train[top_feats]
# y_train = train['<=30Days']
# X_test = test[top_feats]
# y_test = test['<=30Days']

In [100]:
X_train = train.drop(['<=30Days'],axis=1)
y_train = train['<=30Days']
X_test = test.drop(['<=30Days'],axis=1)
y_test = test['<=30Days']

ss = StandardScaler()

X_train = ss.fit_transform(X_train)
X_test = ss.fit_transform(X_test)

#### Stacking

In [55]:
from sklearn.cross_validation import KFold

In [56]:
# Some useful parameters which will come in handy later on
ntrain = train.shape[0]
ntest = test.shape[0]
SEED = 0 # for reproducibility
NFOLDS = 5 # set folds for out-of-fold prediction
kf = KFold(ntrain, n_folds= NFOLDS, random_state=SEED)

# Class to extend the Sklearn classifier
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None):
        #params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)
    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        print(self.clf.fit(x,y).feature_importances_)

In [57]:
def get_oof(clf, x_train, y_train, x_test):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [106]:
knn_params = {
    'n_neighbors': 5,
    'weights': 'distance',
    'p': 1
}

lr_params = {
    'random_state': 0,
    'C': 1,
    'class_weight': 'balanced',
    'penalty': 'l1'
}

rf_params = {
    'random_state': 0,
    'class_weight': 'balanced',
    'criterion': 'gini',
    'max_depth': 50,
    'min_samples_split': 80,
    'n_estimators': 400
}

#### Use 3 Models for 1st stage

In [107]:
rf = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
knn = SklearnHelper(clf=KNeighborsClassifier, seed=SEED, params=knn_params)
lr = SklearnHelper(clf=LogisticRegression, seed=SEED, params=lr_params)

In [108]:
knn_oof_train, knn_oof_test = get_oof(knn, X_train, y_train, X_test)
rf_oof_train, rf_oof_test = get_oof(rf, X_train, y_train, X_test) 
lr_oof_train, lr_oof_test = get_oof(lr, X_train, y_train, X_test)  

#### Add out-of-fold predictions to original train/test

In [358]:
oof_train = train.drop(['<=30Days'],axis=1)
oof_train['knn'] = knn_oof_train.astype(int)
oof_train['rf'] = rf_oof_train.astype(int)
oof_train['lr'] = lr_oof_train.astype(int)

In [359]:
oof_test = test.drop(['<=30Days'],axis=1)
oof_test['knn'] = knn_oof_test.astype(int)
oof_test['rf'] = rf_oof_test.astype(int)
oof_test['lr'] = lr_oof_test.astype(int)

#### Try only preds

In [133]:
oof_train = pd.DataFrame(knn_oof_train.astype(int))
oof_train.columns = ['knn']
oof_train['rf'] = rf_oof_train.astype(int)
oof_train['lr'] = lr_oof_train.astype(int)

In [136]:
oof_test = pd.DataFrame(knn_oof_test.astype(int))
oof_test.columns = ['knn']
oof_test['rf'] = rf_oof_test.astype(int)
oof_test['lr'] = lr_oof_test.astype(int)

#### Only Top Feats

In [366]:
top_feats = ['Age',
 'Dx2_E11',
 'Dx2_E86',
 'Dx2_F03',
 'Dx2_I51',
 'Dx2_T88',
 'Dx2_W19',
 'Dx3_',
 'Dx3_J18',
 'Gender_F',
 'Gender_M',
 'LOS',
 'PastPCPVisits',
 'PrimaryDx_E11',
 'PrimaryDx_I50',
 'PrimaryDx_J44',
 'PrimaryDx_N18']

top_feats = ['Age',
 'Dx2_E11',
 'Dx2_E86',
 'Dx2_F03',
 'Dx2_F05',
 'Dx2_F19',
 'Dx2_G31',
 'Dx2_I51',
 'Dx2_R39',
 'Dx2_T88',
 'Dx2_W19',
 'Dx3_',
 'Dx3_H53',
 'Dx3_J18',
 'Dx3_M54',
 'Dx3_R05',
 'Dx3_R26',
 'Gender_F',
 'Gender_M',
 'LOS',
 'PastPCPVisits',
 'PrimaryDx_A41',
 'PrimaryDx_E11',
 'PrimaryDx_I50',
 'PrimaryDx_J44',
 'PrimaryDx_N18']

In [409]:
oof_train = train[top_feats].copy()
oof_train['knn'] = knn_oof_train.astype(int)
oof_train['rf'] = rf_oof_train.astype(int)
oof_train['lr'] = lr_oof_train.astype(int)
#oof_train['lr'] = oof_train['lr'].map({0:0,1:5})

In [410]:
oof_test = test[top_feats].copy()
oof_test['knn'] = knn_oof_test.astype(int)
oof_test['rf'] = rf_oof_test.astype(int)
oof_test['lr'] = lr_oof_test.astype(int)
#oof_test['lr'] = oof_test['lr'].map({0:0,1:5})

#### Train 2nd stage model

In [350]:
from sklearn.ensemble import AdaBoostClassifier

In [403]:
params = {}
params['criterion'] = ['gini']#,'entropy']
params['n_estimators'] = [1000]#[400,600,800]#[200]
params['max_depth'] = [3,4,5,6,7]#,10,15]#[20,25,30,35,40,45,50,55,60]#[15,20,25]
params['max_features'] = ['auto'] #['auto','sqrt']
#params['min_samples_split'] =  [40,60,80,100] #[2,4,6,8,10]
#params['min_samples_leaf'] = [1,2,3]
params['class_weight'] = ['balanced']

rf = RandomForestClassifier()
stack_grid = GridSearchCV(rf,params,scoring='f1')
stack_grid.fit(oof_train,y_train)

stack = stack_grid.best_estimator_
stack.fit(oof_train,y_train)
stack_pred = stack.predict(oof_test)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'criterion': ['gini'], 'n_estimators': [1000], 'max_depth': [3, 4, 5, 6, 7], 'max_features': ['auto'], 'class_weight': ['balanced']},
       pre_dispatch='2*n_jobs', refit=True, scoring='f1', verbose=0)

In [351]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier

In [399]:
param_grid = {
              "n_estimators": [1000],
              "base_estimator__loss": ['log'],
              "algorithm": ['SAMME'],
             }


LR = LogisticRegression()
SGD = SGDClassifier()

ABC = AdaBoostClassifier(base_estimator = SGD)

# run grid search
grid_search_ABC = GridSearchCV(ABC, param_grid=param_grid,scoring = 'accuracy')
grid_search_ABC.fit(oof_train,y_train)

/Users/xbno/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


GridSearchCV(cv=None, error_score='raise',
       estimator=AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=5, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False),
          learning_rate=1.0, n_estimators=50, random_state=None),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [1000], 'base_estimator__loss': ['log'], 'algorithm': ['SAMME']},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [411]:
ab = AdaBoostClassifier(base_estimator=LogisticRegression(),n_estimators=1000,learning_rate=1)
ab.fit(oof_train,y_train)
stack_pred = ab.predict(oof_test)

#### Analyze the predictions

This is an interesting outcome. Talk through what the precision vs recall mean and calculate them out if there were real numbers involved

In [412]:
preds = pd.DataFrame(y_test)
preds['knn'] = knn_oof_test.astype(int)
preds['lr'] = lr_oof_test.astype(int)
preds['rf'] = rf_oof_test.astype(int)
preds['stack'] = stack_pred

In [413]:
for col in [col for col in preds if '<=30Days' not in col]:
    y_pred = preds[col]
    print('Model: {}'.format(col))
    print('\tprecision: {:.3f}\n\trecall: {:.3f}\n\taccuracy: {:.3f}'.format(precision_score(y_test,y_pred),
                                                       recall_score(y_test,y_pred),
                                                       accuracy_score(y_test,y_pred)))

Model: knn
	precision: 0.433
	recall: 0.133
	accuracy: 0.759
Model: lr
	precision: 0.495
	recall: 0.726
	accuracy: 0.766
Model: rf
	precision: 0.504
	recall: 0.595
	accuracy: 0.771
Model: stack
	precision: 0.591
	recall: 0.367
	accuracy: 0.795


In [415]:
preds[preds['<=30Days'] == 1].mean()

<=30Days    1.000000
knn         0.132548
lr          0.726068
rf          0.594993
stack       0.366716
dtype: float64